In [6]:
import pandas as pd
orders_table = pd.read_csv("../data/olist_orders_dataset.csv")
order_item_table = pd.read_csv("../data/olist_order_items_dataset.csv")

# 1.去除异常数据 
"""
    (2016, 9),   # 2016年9月
    (2016, 10),  # 2016年10月
    (2016, 12),  # 2016年12月
    (2018, 9),   # 2018年9月
    (2018, 10)   # 2018年10月
"""
orders_table['order_purchase_timestamp'] = pd.to_datetime(orders_table['order_purchase_timestamp'],errors='coerce')
orders_table['year'] = orders_table['order_purchase_timestamp'].dt.year
orders_table['month'] = orders_table['order_purchase_timestamp'].dt.month
periods_to_remove = [
    (2016, 9),
    (2016, 10),  # 2016年10月
    (2016, 12),  # 2016年12月
    (2018, 9),   # 2018年9月
    (2018, 10)   # 2018年10月
]
delete_condition = False
for year, month in periods_to_remove:
    delete_condition |= ((orders_table['year'] == year) & (orders_table['month'] == month))
    # 删除指定时间段的数据
orders_table = orders_table[~delete_condition]

# 额外异常检查 由于不存在异常金额（负数金额）后续不进行处理
anomalies = order_item_table[(order_item_table['price']<=0) | (order_item_table['freight_value']<0)]
print("\n 异常金额记录数（price<=0 或 freight<0）：", len(anomalies))



 异常金额记录数（price<=0 或 freight<0）： 0


In [7]:

# 2.检查金额异常值
for col in ['price','freight_value']:
    desc = order_item_table[col].describe(percentiles=[.01,.25,.5,.75,.95,.99])
    print(f"\n {col} 分布统计：\n", desc)

# 2.1去掉极端值
p99_price = order_item_table['price'].quantile(0.99)
order_item_table.loc[order_item_table['price'] > p99_price, 'price'] = p99_price
p99_freight_value = order_item_table['freight_value'].quantile(0.99)
order_item_table.loc[order_item_table['freight_value'] > p99_freight_value, 'freight_value'] = p99_freight_value


 price 分布统计：
 count    112650.000000
mean        120.653739
std         183.633928
min           0.850000
1%            9.990000
25%          39.900000
50%          74.990000
75%         134.900000
95%         349.900000
99%         890.000000
max        6735.000000
Name: price, dtype: float64

 freight_value 分布统计：
 count    112650.000000
mean         19.990320
std          15.806405
min           0.000000
1%            4.419800
25%          13.080000
50%          16.260000
75%          21.150000
95%          45.120000
99%          84.520000
max         409.680000
Name: freight_value, dtype: float64


In [8]:
# 3.检查关联完整性
# items无订单
missing_orders = order_item_table[~order_item_table['order_id'].isin(orders_table['order_id'])]
print("\n items中找不到对应订单的数量：", len(missing_orders))
if len(missing_orders) > 0:
    order_item_table = order_item_table[order_item_table['order_id'].isin(orders_table['order_id'])]

# 订单无明细    
missing_items = orders_table[~orders_table['order_id'].isin(order_item_table['order_id'])]
print(" orders中没有明细的订单数量：", len(missing_items))
if len(missing_items) > 0:
    orders_table = orders_table[orders_table['order_id'].isin(order_item_table['order_id'])]



 items中找不到对应订单的数量： 371
 orders中没有明细的订单数量： 739


In [9]:

# 4.验证订单状态分布
status_check = orders_table['order_status'].value_counts(normalize=True).mul(100).round(2)
status_count = orders_table['order_status'].value_counts()
status_dis = pd.DataFrame({
    "count":status_count,
    "pct":status_check
})
print("订单状态分别情况",status_dis)

# 5.筛选delivered订单
cond1 = orders_table['order_status'] == 'delivered'
orders  =  orders_table.loc[cond1,:]
orders_table =  orders_table.loc[cond1,['order_id','order_status','order_purchase_timestamp','order_delivered_customer_date']]



订单状态分别情况               count    pct
order_status              
delivered     96211  97.82
shipped        1097   1.12
canceled        448   0.46
processing      299   0.30
invoiced        296   0.30
approved          2   0.00


In [ ]:

# 6.构建gmv_value字段
order_item_table['gmv_value'] = order_item_table['price'] + order_item_table['freight_value']
items = order_item_table.copy()
order_items = order_item_table[['order_id','gmv_value']]


order_item_wide = orders_table.merge(order_items,on='order_id',how='inner')
order_item_wide.to_csv("../processed/order_item_wide_gmv_cleaned.csv",index=False)